<h1> Segmenting and Clustering Neighborhoods in Toronto</h1>

## Introduction

<i> This code identifies neighborhood area segments in Toronto and cluster them according to venues available in vicinity of those neighborhoods.
    
There are 3 steps -
1. Download and Stucture Toronto Data
2. Explore Toronto Neighbourhoods
3. Analyse and Clusture Neighborhoods </i> 

<u> Note: Here Only 1st two steps are covered.</u>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

! pip install beautifulsoup4 # This library helps parsing webpages data 
from bs4 import BeautifulSoup

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

! pip install geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

    100% |████████████████████████████████| 102kB 16.9MB/s 
Libraries imported.


<i> Download Toronto Neighborhood Data (Tip: Zip code starts from 'M') </i>

In [2]:
!wget -q -O 'toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Web data downloaded!')

Web data downloaded!


<i> Parse the web data into meaningful information </i> 

In [3]:
with open('toronto_data.html') as html_file:
    soup=BeautifulSoup(html_file,'lxml')
table_html=soup.find('table',class_='wikitable sortable')
#print(table_html)

<i> Create dataframe to read web data for toronto postal codes </i>

In [4]:
col_names=['PostalCode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns=col_names)
neighborhoods

,PostalCode,Borough,Neighborhood


In [5]:
i=0
j=0
for tr in table_html.tbody.find_all('tr'):
    if i==0:
        i=i+1
    else:  
        for td in tr.find_all('td'):
            if j==0:
                postalcode_cd = td.text
                j=j+1
            elif j==1:
                borough_name = td.text
                j=j+1
            else:
                neighborhood_name = td.text
                j=0
                neighborhoods = neighborhoods.append({'PostalCode': postalcode_cd,
                                                      'Borough': borough_name,       
                                                      'Neighborhood': neighborhood_name},                                         
                                                       ignore_index=True)

print('Size',neighborhoods.shape)
neighborhoods.head()

Size (288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [6]:
# Remove "\n" from Neighborhood column text
for index, row in neighborhoods.iterrows():
    row['Neighborhood']=row['Neighborhood'].rstrip()

# Drop rows where Borough not assigned    
neighborhoods = neighborhoods[neighborhoods.Borough != "Not assigned"]
neighborhoods = neighborhoods.reset_index(drop=True)

# Assign Borough value where neighborhood not assigned  
for index, row in neighborhoods.iterrows():
    if row.at['Neighborhood'] == "Not assigned":
            row.at['Neighborhood'] = row.at['Borough']
            
print('Size',neighborhoods.shape)
neighborhoods.head()

Size (211, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [7]:
# Combine Neighbourhood location for same Postal code 
neighborhoods = neighborhoods.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print('Size',neighborhoods.shape)
neighborhoods.head(15)

Size (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
neighborhoods.shape

(103, 3)

<i> Use geocoder library to get the latitude and longitude values of Toronto City </i>

In [9]:
import geocoder

def get_geocoder_latlong(postal_code):
    # initialize variable to None
    lat_long_coords = None
    # loop until it gets the coordinates
    while(lat_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_long_coords = g.latlng
    return lat_long_coords

<i> Populate Latitide and Longitude based on Toronto City Postal Codes </i>

In [10]:
p_codes = neighborhoods['PostalCode']    
coords = [ get_geocoder_latlong(postal_code) for postal_code in p_codes.tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
neighborhoods['Latitude'] = df_coords['Latitude']
neighborhoods['Longitude'] = df_coords['Longitude']

In [11]:
neighborhoods.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163


<i> Create a map to see how boroughs are spread in Toronto </i>  

In [12]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [13]:
# map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
#for lat, lng, borough, neighborhood in range(0,len(neighborhoods)):
    label = '{}, {}'.format(borough,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<i> Now, let us analyze areas where borough names contains word 'Toronto' </i>   

In [14]:
borough_toronto = neighborhoods[neighborhoods['Borough'].str.contains("Toronto") == True].reset_index(drop=True) 
borough_toronto.shape

(38, 5)

In [15]:
borough_toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
7,M4S,Central Toronto,Davisville,43.703395,-79.385964
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690655,-79.383561
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686083,-79.402335


<i> Let us see in map how boroughs are distributed in Toronto </i>

In [16]:
# map of Toronto using latitude and longitude values
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(borough_toronto['Latitude'], borough_toronto['Longitude'], borough_toronto['Borough'], borough_toronto['Neighborhood']):
    label = '{}, {}'.format(borough,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

<i> Now, let us explore neighbourhoods and segment them according to popular venues nearby </i>

In [17]:
# Let us use Foursquare
CLIENT_ID = '42DAMOEH2K4BYCYRCA4HDXVG3UTDATXMKC4QPTB5URHHDJX5' # your Foursquare ID
CLIENT_SECRET = 'GFXHWBZWSJAVACDII3OJP4MVVA4RSHJDCZEGGFSU15EAY13I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

<i> Define function to get nearby venues and categories </i>

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

<i> Now, let us apply the function to get toronto neighborhood nearby top 20 veneues </i>

In [37]:
LIMIT = 20 
toronto_venues = getNearbyVenues(names=borough_toronto['Neighborhood'],
                                   latitudes=borough_toronto['Latitude'],
                                   longitudes=borough_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [38]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
0,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,Health Food Store
1,The Beaches,43.676845,-79.295225,Grover Pub and Grub,Pub
2,The Beaches,43.676845,-79.295225,Upper Beaches,Neighborhood
3,"The Danforth West, Riverdale",43.683262,-79.355120,Dollarama,Discount Store
4,"The Danforth West, Riverdale",43.683262,-79.355120,Sobeys Urban Fresh,Grocery Store


<i> Let's check how many venues were returned for each neighborhood </i>

In [39]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
Neighborhood,,,,
"Adelaide, King, Richmond",20,20,20,20
Berczy Park,20,20,20,20
"Brockton, Exhibition Place, Parkdale Village",20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,20,20,20,20
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",20,20,20,20
"Cabbagetown, St. James Town",20,20,20,20
Central Bay Street,20,20,20,20
"Chinatown, Grange Park, Kensington Market",20,20,20,20
Christie,10,10,10,10


<i> Let's find out how many unique categories can be curated from all the returned venues </i>

In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 148 uniques categories.


<i> Let us analyze each neighborhood </i>

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# get neighborhood column position
col_pos = toronto_onehot.columns.get_loc('Neighborhood')

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[col_pos]] + list(toronto_onehot.columns[:col_pos])+ list(toronto_onehot.columns[col_pos+1:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
toronto_onehot.shape

(554, 148)

<i> Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category </i>

In [48]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Beer Bar,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.100000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.050000,0.000000,0.050000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.050000,0.00,0.00,0.00,0.000000,0.0,0.100000,0.00,0.00,0.00,0.05,0.000000,0.000000,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.05,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.050000,0.050000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.05,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.05,0.00,0.00,0.05,0.00,0.000000,0.00,0.00,0.00,0.00,0.10,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00
1,Berczy Park,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.05,0.00,0.00,0.000000,0.050000,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.050000,0.00,0.00,0.00,0.000000,0.1,0.000000,0.00,0.00,0.00,0.05,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.10,0.000000,0.00,0.05,0.00,0.00,0.000000,0.00,0.05,0.05,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.050000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.05,0.000000,0.00,0.00,0.00,0.050000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.000000,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.000000,0.00,0.00,0.05,0.00,0.05,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.050000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.050000,0.00,0.00,0.0,0.0

In [49]:
toronto_grouped.shape

(37, 148)

<i> Let us create a function to sort venues in decending order </i>

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<i> Now let's create the new dataframe and display the top 5 venues for each neighborhood </i>

In [103]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",American Restaurant,Steakhouse,Coffee Shop,Bakery,Seafood Restaurant
1,Berczy Park,Cocktail Bar,Seafood Restaurant,Farmers Market,Park,Food Truck
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Italian Restaurant,Furniture / Home Store,Bakery,Sandwich Place
3,Business Reply Mail Processing Centre 969 Eastern,Concert Hall,Café,Burrito Place,Hotel,Pizza Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Italian Restaurant,Yoga Studio,Park,Café,Sandwich Place


<i> Now, let us use k-means method to cluster neighborhood </i>

In [104]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([0, 0, 0, 0, 0], dtype=int32)

<i> Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood </i>

In [105]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = borough_toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [108]:
toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int) 
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,0,Health Food Store,Pub,Yoga Studio,Dumpling Restaurant,Farmers Market
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,0,Grocery Store,Discount Store,Bus Line,Park,Dance Studio
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673,0,Sandwich Place,Park,Movie Theater,Fish & Chips Shop,Burrito Place
3,M4M,East Toronto,Studio District,43.662766,-79.334830,0,Coffee Shop,Brewery,American Restaurant,Bar,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,1,Construction & Landscaping,Bus Line,Swim School,Eastern European Restaurant,Farmers Market
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526,0,American Restaurant,Hotel,Clothing Store,Breakfast Spot,Food & Drink Shop
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960,4,Park,Playground,Gym Pool,Garden,Discount Store
7,M4S,Central Toronto,Davisville,43.703395,-79.385964,0,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Sandwich Place
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690655,-79.383561,4,Tennis Court,Gym,Trail,Playground,Diner
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686083,-79.402335,0,Light Rail Station,Coffee Shop,Convenience Store,Supermarket,Eastern European Restaurant


<i> Let's visualize the resulting clusters </i>

In [113]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='white',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters